# Amsterdam Weather Analysis with Conditional Mixture Models

This notebook analyzes hourly weather data from Amsterdam using conditional Gaussian mixture models to understand the relationships between temperature, wind speed, and solar radiation over time.

**Key Questions:**
- How do weather variables relate to seasonal and daily cycles?
- What are the typical weather patterns at different times of year?
- How well can we model the joint distribution of weather variables?

**Data:** Hourly weather data from Amsterdam (2014-2023) including temperature, wind speed, and solar radiation.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

from cgmm import ConditionalGMMRegressor, MixtureOfExpertsRegressor, DiscriminativeConditionalGMMRegressor

# Set style for better plots
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
np.random.seed(42)


## Data Loading and Preprocessing

Load the Amsterdam weather data and prepare it for analysis. We'll focus on three key variables:
- **Temperature** (`temp_c`): Air temperature in Celsius
- **Wind Speed** (`wind_ms`): Wind speed in m/s (log-transformed)
- **Solar Radiation** (`ghi_wm2`): Global horizontal irradiance in W/m² (log-transformed)

The conditioning variables will be derived from the datetime to capture seasonal and daily patterns.


In [2]:
# Load the weather data
df = pd.read_csv('data/amsterdam_hourly.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"\nMissing values:")
print(df[['temp_c', 'wind_ms', 'ghi_wm2']].isnull().sum())

# Remove rows with missing values
df_clean = df[['datetime', 'temp_c', 'wind_ms', 'ghi_wm2']].dropna()
print(f"\nClean dataset shape: {df_clean.shape}")

# Display basic statistics
print("\nBasic statistics:")
print(df_clean[['temp_c', 'wind_ms', 'ghi_wm2']].describe())


Dataset shape: (52608, 7)
Date range: 2019-01-01 00:00:00 to 2024-12-31 23:00:00

Missing values:
temp_c     0
wind_ms    0
ghi_wm2    0
dtype: int64

Clean dataset shape: (52608, 4)

Basic statistics:
             temp_c       wind_ms       ghi_wm2
count  52608.000000  52608.000000  52608.000000
mean      11.327055     15.325150    128.068602
std        6.339285      7.743829    198.913009
min      -10.300000      0.000000      0.000000
25%        6.700000      9.400000      0.000000
50%       10.900000     14.100000      5.000000
75%       15.900000     19.700000    195.000000
max       38.000000     61.400000    869.000000


In [3]:
# Create time-based features for conditioning
df_clean['year'] = df_clean['datetime'].dt.year
df_clean['month'] = df_clean['datetime'].dt.month
df_clean['day_of_year'] = df_clean['datetime'].dt.dayofyear
df_clean['hour'] = df_clean['datetime'].dt.hour

# Create cyclical features
# Annual cycle (seasonal)
df_clean['annual_sin'] = np.sin(2 * np.pi * df_clean['day_of_year'] / 365.25)
df_clean['annual_cos'] = np.cos(2 * np.pi * df_clean['day_of_year'] / 365.25)

# Daily cycle (diurnal)
df_clean['daily_sin'] = np.sin(2 * np.pi * df_clean['hour'] / 24)
df_clean['daily_cos'] = np.cos(2 * np.pi * df_clean['hour'] / 24)

print("Created cyclical features:")
print("- Annual cycle: annual_sin, annual_cos")
print("- Daily cycle: daily_sin, daily_cos")


Created cyclical features:
- Annual cycle: annual_sin, annual_cos
- Daily cycle: daily_sin, daily_cos


In [4]:
# Transform positive variables using log(1+x) to handle zeros
df_clean['wind_ms_log'] = np.log1p(df_clean['wind_ms'])  # log(1 + wind_ms)
df_clean['ghi_wm2_log'] = np.log1p(df_clean['ghi_wm2'])  # log(1 + ghi_wm2)

# Prepare target variables (weather variables)
targets = ['temp_c', 'wind_ms_log', 'ghi_wm2_log']
y = df_clean[targets].values

# Prepare conditioning variables (time-based features)
conditioning_vars = ['annual_sin', 'annual_cos', 'daily_sin', 'daily_cos']
X = df_clean[conditioning_vars].values

print(f"Target variables: {targets}")
print(f"Conditioning variables: {conditioning_vars}")
print(f"X shape: {X.shape}, y shape: {y.shape}")

# Show transformation effect
print("\nTransformation comparison:")
print("Wind speed - Original vs Log-transformed:")
print(f"  Original: mean={df_clean['wind_ms'].mean():.2f}, std={df_clean['wind_ms'].std():.2f}")
print(f"  Log(1+x): mean={df_clean['wind_ms_log'].mean():.2f}, std={df_clean['wind_ms_log'].std():.2f}")
print("\nSolar radiation - Original vs Log-transformed:")
print(f"  Original: mean={df_clean['ghi_wm2'].mean():.2f}, std={df_clean['ghi_wm2'].std():.2f}")
print(f"  Log(1+x): mean={df_clean['ghi_wm2_log'].mean():.2f}, std={df_clean['ghi_wm2_log'].std():.2f}")


Target variables: ['temp_c', 'wind_ms_log', 'ghi_wm2_log']
Conditioning variables: ['annual_sin', 'annual_cos', 'daily_sin', 'daily_cos']
X shape: (52608, 4), y shape: (52608, 3)

Transformation comparison:
Wind speed - Original vs Log-transformed:
  Original: mean=15.33, std=7.74
  Log(1+x): mean=2.68, std=0.50

Solar radiation - Original vs Log-transformed:
  Original: mean=128.07, std=198.91
  Log(1+x): mean=2.55, std=2.63


## Discriminative Model Investigation

Systematic investigation of the Discriminative Conditional GMM Regressor on the FULL dataset with various hyperparameter settings.


In [5]:
# Define hyperparameter grid for systematic testing
hyperparameter_grid = [
    # (n_components, max_iter, tol, weight_step)
    (3, 100, 1e-3, 1),
    (5, 100, 1e-3, 1),
    (10, 100, 1e-3, 1),
    (15, 100, 1e-3, 1),

    (3, 200, 1e-4, 0.5),
    (3, 300, 1e-5, 0.1),
    (3, 500, 1e-6, 0.05),
    (5, 100, 1e-3, 1),
    (5, 200, 1e-4, 0.5),
    (5, 300, 1e-5, 0.1),
    (5, 500, 1e-6, 0.05),
    (8, 100, 1e-3, 1),
    (8, 200, 1e-4, 0.5),
    (8, 300, 1e-5, 0.1),
    (8, 500, 1e-6, 0.05),
    (10, 200, 1e-4, 5),
    (10, 300, 1e-5, 1),
    (10, 500, 1e-6, 0.5),
    (15, 200, 1e-4, 5),
    (15, 300, 1e-5, 1),
    (15, 500, 1e-6, 0.5)
]

print(f"Testing {len(hyperparameter_grid)} hyperparameter combinations on FULL dataset")
print(f"Dataset size: X={X.shape}, y={y.shape}")
print("\nHyperparameter combinations:")
for i, (n_comp, max_iter, tol, weight_step) in enumerate(hyperparameter_grid):
    print(f"  {i+1:2d}. n_components={n_comp:2d}, max_iter={max_iter:3d}, tol={tol:.0e}, weight_step={weight_step:.3f}")


Testing 21 hyperparameter combinations on FULL dataset
Dataset size: X=(52608, 4), y=(52608, 3)

Hyperparameter combinations:
   1. n_components= 3, max_iter=100, tol=1e-03, weight_step=1.000
   2. n_components= 5, max_iter=100, tol=1e-03, weight_step=1.000
   3. n_components=10, max_iter=100, tol=1e-03, weight_step=1.000
   4. n_components=15, max_iter=100, tol=1e-03, weight_step=1.000
   5. n_components= 3, max_iter=200, tol=1e-04, weight_step=0.500
   6. n_components= 3, max_iter=300, tol=1e-05, weight_step=0.100
   7. n_components= 3, max_iter=500, tol=1e-06, weight_step=0.050
   8. n_components= 5, max_iter=100, tol=1e-03, weight_step=1.000
   9. n_components= 5, max_iter=200, tol=1e-04, weight_step=0.500
  10. n_components= 5, max_iter=300, tol=1e-05, weight_step=0.100
  11. n_components= 5, max_iter=500, tol=1e-06, weight_step=0.050
  12. n_components= 8, max_iter=100, tol=1e-03, weight_step=1.000
  13. n_components= 8, max_iter=200, tol=1e-04, weight_step=0.500
  14. n_componen

In [ ]:
# Test all hyperparameter combinations on FULL dataset
results = []

print("\n=== DISCRIMINATIVE MODEL HYPERPARAMETER TESTING ===")
print("Testing on FULL dataset with consistent evaluation")
print("\nProgress:")

for i, (n_components, max_iter, tol, weight_step) in enumerate(hyperparameter_grid):
    print(f"\n{i+1:2d}/{len(hyperparameter_grid)}: n_comp={n_components:2d}, max_iter={max_iter:3d}, tol={tol:.0e}, weight_step={weight_step:.3f}")

    # Create model with current hyperparameters
    model = DiscriminativeConditionalGMMRegressor(
        n_components=n_components,
        max_iter=max_iter,
        tol=tol,
        weight_step=weight_step,
        random_state=42
    )   
    
    # Train on FULL dataset
    model.fit(X, y)
    
    # Evaluate on FULL dataset (consistent evaluation)
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    log_likelihood = model.score(X, y)  # This is the key metric!
    
    # Get convergence info
    iterations = getattr(model, 'n_iter_', 'Unknown')
    converged = getattr(model, 'converged_', 'Unknown')
    
    # Store results
    result = {
        'n_components': n_components,
        'max_iter': max_iter,
        'tol': tol,
        'weight_step': weight_step,
        'iterations': iterations,
        'converged': converged,
        'mse': mse,
        'r2': r2,
        'log_likelihood': log_likelihood
    }
    results.append(result)
    
    # Print results
    print(f"    Iterations: {iterations}, Converged: {converged}")
    print(f"    MSE: {mse:.4f}, R²: {r2:.4f}, Log-likelihood: {log_likelihood:.4f}")
    
    # Check for issues
    if np.isnan(log_likelihood) or np.isinf(log_likelihood):
        print(f"    WARNING: Invalid log-likelihood!")
    if iterations == max_iter:
        print(f"    WARNING: Hit max_iter limit!")


print("\n=== TESTING COMPLETED ===")



=== DISCRIMINATIVE MODEL HYPERPARAMETER TESTING ===
Testing on FULL dataset with consistent evaluation

Progress:

 1/21: n_comp= 3, max_iter=100, tol=1e-03, weight_step=1.000


In [ ]:
# Analyze results
results_df = pd.DataFrame(results)

print("\n=== DISCRIMINATIVE MODEL RESULTS ANALYSIS ===")
print(f"Successful runs: {len(results_df.dropna())}/{len(results_df)}")

# Sort by log-likelihood (best first)
valid_results = results_df.dropna().copy()
valid_results = valid_results.sort_values('log_likelihood', ascending=False)

print("\nTop 10 configurations by log-likelihood:")
print(valid_results[['n_components', 'max_iter', 'tol', 'weight_step', 'iterations', 'converged', 'log_likelihood', 'r2']].head(10).to_string(index=False))

print("\nWorst 5 configurations by log-likelihood:")
print(valid_results[['n_components', 'max_iter', 'tol', 'weight_step', 'iterations', 'converged', 'log_likelihood', 'r2']].tail(5).to_string(index=False))

# Summary statistics
print(f"\nLog-likelihood statistics:")
print(f"  Mean: {valid_results['log_likelihood'].mean():.4f}")
print(f"  Std:  {valid_results['log_likelihood'].std():.4f}")
print(f"  Min:  {valid_results['log_likelihood'].min():.4f}")
print(f"  Max:  {valid_results['log_likelihood'].max():.4f}")

print(f"\nIteration statistics:")
print(f"  Mean: {valid_results['iterations'].mean():.1f}")
print(f"  Std:  {valid_results['iterations'].std():.1f}")
print(f"  Min:  {valid_results['iterations'].min()}")
print(f"  Max:  {valid_results['iterations'].max()}")

print(f"\nConvergence rate: {(valid_results['converged'] == True).mean():.1%}")


In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Log-likelihood vs n_components
axes[0, 0].scatter(valid_results['n_components'], valid_results['log_likelihood'], alpha=0.7)
axes[0, 0].set_xlabel('Number of Components')
axes[0, 0].set_ylabel('Log-likelihood')
axes[0, 0].set_title('Log-likelihood vs Components')
axes[0, 0].grid(True, alpha=0.3)

# Log-likelihood vs iterations
axes[0, 1].scatter(valid_results['iterations'], valid_results['log_likelihood'], alpha=0.7)
axes[0, 1].set_xlabel('Iterations')
axes[0, 1].set_ylabel('Log-likelihood')
axes[0, 1].set_title('Log-likelihood vs Iterations')
axes[0, 1].grid(True, alpha=0.3)

# Log-likelihood vs tolerance
axes[0, 2].scatter(valid_results['tol'], valid_results['log_likelihood'], alpha=0.7)
axes[0, 2].set_xlabel('Tolerance')
axes[0, 2].set_ylabel('Log-likelihood')
axes[0, 2].set_title('Log-likelihood vs Tolerance')
axes[0, 2].set_xscale('log')
axes[0, 2].grid(True, alpha=0.3)

# Log-likelihood vs weight_step
axes[1, 0].scatter(valid_results['weight_step'], valid_results['log_likelihood'], alpha=0.7)
axes[1, 0].set_xlabel('Weight Step')
axes[1, 0].set_ylabel('Log-likelihood')
axes[1, 0].set_title('Log-likelihood vs Weight Step')
axes[1, 0].set_xscale('log')
axes[1, 0].grid(True, alpha=0.3)

# R² vs log-likelihood
axes[1, 1].scatter(valid_results['log_likelihood'], valid_results['r2'], alpha=0.7)
axes[1, 1].set_xlabel('Log-likelihood')
axes[1, 1].set_ylabel('R²')
axes[1, 1].set_title('R² vs Log-likelihood')
axes[1, 1].grid(True, alpha=0.3)

# Iterations vs tolerance
axes[1, 2].scatter(valid_results['tol'], valid_results['iterations'], alpha=0.7)
axes[1, 2].set_xlabel('Tolerance')
axes[1, 2].set_ylabel('Iterations')
axes[1, 2].set_title('Iterations vs Tolerance')
axes[1, 2].set_xscale('log')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Train the best model and analyze it in detail
best_config = valid_results.iloc[0]
print(f"\n=== BEST MODEL ANALYSIS ===")
print(f"Best configuration:")
print(f"  n_components: {best_config['n_components']}")
print(f"  max_iter: {best_config['max_iter']}")
print(f"  tol: {best_config['tol']}")
print(f"  weight_step: {best_config['weight_step']}")
print(f"  Performance: log_likelihood={best_config['log_likelihood']:.4f}, r2={best_config['r2']:.4f}")

# Train the best model
best_model = DiscriminativeConditionalGMMRegressor(
    n_components=int(best_config['n_components']),
    max_iter=int(best_config['max_iter']),
    tol=best_config['tol'],
    weight_step=best_config['weight_step'],
    random_state=42
)

print(f"\nTraining best model on FULL dataset...")
best_model.fit(X, y)

# Final evaluation
y_pred_best = best_model.predict(X)
mse_best = mean_squared_error(y, y_pred_best)
r2_best = r2_score(y, y_pred_best)
log_likelihood_best = best_model.score(X, y)

print(f"\nFinal performance on FULL dataset:")
print(f"  MSE: {mse_best:.4f}")
print(f"  R²: {r2_best:.4f}")
print(f"  Log-likelihood: {log_likelihood_best:.4f}")
print(f"  Iterations: {getattr(best_model, 'n_iter_', 'Unknown')}")
print(f"  Converged: {getattr(best_model, 'converged_', 'Unknown')}")

# Check prediction ranges
print(f"\nPrediction analysis:")
print(f"  Target range: [{y.min():.3f}, {y.max():.3f}]")
print(f"  Prediction range: [{y_pred_best.min():.3f}, {y_pred_best.max():.3f}]")
print(f"  Target mean: {y.mean():.3f}")
print(f"  Prediction mean: {y_pred_best.mean():.3f}")

# Check for any issues
nan_pred = np.isnan(y_pred_best).sum()
inf_pred = np.isinf(y_pred_best).sum()
if nan_pred > 0:
    print(f"  WARNING: {nan_pred} NaN predictions!")
if inf_pred > 0:
    print(f"  WARNING: {inf_pred} infinite predictions!")

print(f"\n=== BEST MODEL ANALYSIS COMPLETE ===")


In [ ]:
# Final comparison: Fixed Discriminative vs Conditional GMM
print("\n=== FINAL PERFORMANCE COMPARISON ===")

# Use a larger subset for meaningful comparison
X_final = X[:1000]
y_final = y[:1000]

print(f"Final comparison on subset: X={X_final.shape}, y={y_final.shape}")

# Train Conditional GMM
conditional_final = ConditionalGMMRegressor(
    n_components=3,
    random_state=42
)
conditional_final.fit(X_final, y_final)

# Train Discriminative (using corrected source code)
discriminative_final = DiscriminativeConditionalGMMRegressor(
    n_components=3,
    max_iter=100,
    tol=1e-4,
    weight_step=0.01,
    random_state=42
)
discriminative_final.fit(X_final, y_final)

# Compare log-likelihoods
conditional_ll_final = conditional_final.score(X_final, y_final)
discriminative_ll_final = discriminative_final.score(X_final, y_final)

print(f"\nLog-likelihood comparison:")
print(f"  Conditional GMM: {conditional_ll_final:.6f}")
print(f"  Discriminative: {discriminative_ll_final:.6f}")
print(f"  Difference (Conditional - Discriminative): {conditional_ll_final - discriminative_ll_final:.6f}")

# Compare R² scores
conditional_pred_final = conditional_final.predict(X_final)
discriminative_pred_final = discriminative_final.predict(X_final)

conditional_r2_final = r2_score(y_final, conditional_pred_final)
discriminative_r2_final = r2_score(y_final, discriminative_pred_final)

print(f"\nR² comparison:")
print(f"  Conditional GMM: {conditional_r2_final:.6f}")
print(f"  Discriminative: {discriminative_r2_final:.6f}")
print(f"  Difference (Conditional - Discriminative): {conditional_r2_final - discriminative_r2_final:.6f}")



In [ ]:
# CORRECTED: Test the properly fixed Discriminative model
print("\n=== TESTING CORRECTED LOG_PROB IMPLEMENTATION ===")

# Test on a small subset
X_test_corrected = X[:100]
y_test_corrected = y[:100]

print(f"Testing corrected implementation on subset: X={X_test_corrected.shape}, y={y_test_corrected.shape}")

# Create the CORRECTED Discriminative model
corrected_discriminative = DiscriminativeConditionalGMMRegressor(
    n_components=3,
    max_iter=50,
    tol=1e-4,
    weight_step=0.01,
    random_state=42
)

# Train the model
corrected_discriminative.fit(X_test_corrected, y_test_corrected)

# Test consistency between training and evaluation
internal_ll = corrected_discriminative._mean_conditional_loglik(X_test_corrected, y_test_corrected)
score_ll = corrected_discriminative.score(X_test_corrected, y_test_corrected)
log_prob_ll = corrected_discriminative.log_prob(X_test_corrected, y_test_corrected)
log_prob_mean = np.mean(log_prob_ll)

print(f"\nCorrected implementation results:")
print(f"  Internal log-likelihood (training): {internal_ll:.6f}")
print(f"  Score method log-likelihood: {score_ll:.6f}")
print(f"  Log_prob method mean: {log_prob_mean:.6f}")
print(f"  Difference (internal - score): {internal_ll - score_ll:.6f}")
print(f"  Difference (internal - log_prob): {internal_ll - log_prob_mean:.6f}")

if abs(internal_ll - score_ll) < 1e-6:
    print("✅ SUCCESS: Corrected implementation - log-likelihood calculations are now consistent!")
    print("The Discriminative model now correctly computes log p(y|x) for evaluation.")
else:
    print("❌ Still inconsistent after correction.")

print(f"\n=== KEY INSIGHT ===")
print("The Discriminative model's log_prob() should return log p(y|x), not the discriminative EM objective.")
print("The discriminative EM objective (log p(x,y) - log p(x)) is only used during training.")
print("For evaluation, we want log p(y|x) which is what the base class correctly computes.")


# Amsterdam Weather Analysis with Conditional Mixture Models

This notebook analyzes hourly weather data from Amsterdam using conditional Gaussian mixture models to understand the relationships between temperature, wind speed, and solar radiation over time.

**Key Questions:**
- How do weather variables relate to seasonal and daily cycles?
- What are the typical weather patterns at different times of year?
- How well can we model the joint distribution of weather variables?

**Data:** Hourly weather data from Amsterdam (2014-2023) including temperature, wind speed, and solar radiation.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

from cgmm import ConditionalGMMRegressor, MixtureOfExpertsRegressor, DiscriminativeConditionalGMMRegressor

# Set style for better plots
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
np.random.seed(42)


## Data Loading and Preprocessing

Load the Amsterdam weather data and prepare it for analysis. We'll focus on three key variables:
- **Temperature** (`temp_c`): Air temperature in Celsius
- **Wind Speed** (`wind_ms`): Wind speed in m/s (log-transformed)
- **Solar Radiation** (`ghi_wm2`): Global horizontal irradiance in W/m² (log-transformed)

The conditioning variables will be derived from the datetime to capture seasonal and daily patterns.


In [ ]:
# Load the weather data
df = pd.read_csv('data/amsterdam_hourly.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"\nMissing values:")
print(df[['temp_c', 'wind_ms', 'ghi_wm2']].isnull().sum())

# Remove rows with missing values
df_clean = df[['datetime', 'temp_c', 'wind_ms', 'ghi_wm2']].dropna()
print(f"\nClean dataset shape: {df_clean.shape}")

# Display basic statistics
print("\nBasic statistics:")
print(df_clean[['temp_c', 'wind_ms', 'ghi_wm2']].describe())


In [ ]:
# DEBUG: Let's investigate why all hyperparameters give identical results
print("\n=== DEBUGGING IDENTICAL RESULTS ===")

# Test just 3 different hyperparameter combinations
debug_combinations = [
    (3, 50, 1e-3, 0.1),   # Low iterations, high tolerance, high weight_step
    (3, 200, 1e-6, 0.01), # High iterations, low tolerance, low weight_step  
    (5, 100, 1e-4, 0.05)  # Different n_components
]

print("Testing 3 different hyperparameter combinations:")
for i, (n_components, max_iter, tol, weight_step) in enumerate(debug_combinations):
    print(f"\n{i+1}. n_comp={n_components}, max_iter={max_iter}, tol={tol:.0e}, weight_step={weight_step:.3f}")
    
    # Create model
    model = DiscriminativeConditionalGMMRegressor(
        n_components=n_components,
        max_iter=max_iter,
        tol=tol,
        weight_step=weight_step,
        random_state=42
    )
    
    # Train on small subset for debugging
    X_debug = X[:100]
    y_debug = y[:100]
    model.fit(X_debug, y_debug)
    
    # Check if parameters actually changed
    print(f"  Final weights: {model.weights_}")
    print(f"  Final means shape: {model.means_.shape}")
    print(f"  Final means[0]: {model.means_[0]}")
    print(f"  Iterations: {model.n_iter_}, Converged: {model.converged_}")
    
    # Check log-likelihood
    ll = model.score(X_debug, y_debug)
    print(f"  Log-likelihood: {ll:.6f}")
    
    # Check if the model is actually different
    if i > 0:
        prev_weights = debug_combinations[i-1][0]
        if prev_weights == n_components:
            print(f"  Same n_components as previous - weights should be different")
        else:
            print(f"  Different n_components - should have different shape")

print(f"\n=== DIAGNOSIS ===")
print("If all results are identical, the issue could be:")
print("1. The discriminative EM algorithm is not actually updating parameters")
print("2. The convergence criteria is too strict and stopping immediately")
print("3. The random_state is making all models identical")
print("4. The log_prob method has a bug")


In [ ]:
# CRITICAL DEBUG: Let's check if the models are actually different
print("\n=== CRITICAL DEBUG: CHECKING IF MODELS ARE ACTUALLY DIFFERENT ===")

# Test the exact same hyperparameters from your output
test_combinations = [
    (3, 100, 1e-3, 0.1),   # Should give 5 iterations
    (3, 200, 1e-4, 0.05),  # Should give 6 iterations
]

for i, (n_components, max_iter, tol, weight_step) in enumerate(test_combinations):
    print(f"\n=== TESTING COMBINATION {i+1} ===")
    print(f"n_comp={n_components}, max_iter={max_iter}, tol={tol:.0e}, weight_step={weight_step:.3f}")
    
    # Create model
    model = DiscriminativeConditionalGMMRegressor(
        n_components=n_components,
        max_iter=max_iter,
        tol=tol,
        weight_step=weight_step,
        random_state=42
    )
    
    # Train on small subset
    X_test = X[:100]
    y_test = y[:100]
    model.fit(X_test, y_test)
    
    # Check if parameters are actually different
    print(f"Final weights: {model.weights_}")
    print(f"Final means[0]: {model.means_[0]}")
    print(f"Final covariances[0] shape: {model.covariances_[0].shape}")
    print(f"Final covariances[0] diagonal: {np.diag(model.covariances_[0])}")
    
    # Check predictions
    y_pred = model.predict(X_test)
    print(f"Prediction range: [{y_pred.min():.6f}, {y_pred.max():.6f}]")
    print(f"Prediction mean: {y_pred.mean():.6f}")
    print(f"Prediction std: {y_pred.std():.6f}")
    
    # Check log-likelihood
    ll = model.score(X_test, y_test)
    print(f"Log-likelihood: {ll:.6f}")
    
    # Store for comparison
    if i == 0:
        prev_weights = model.weights_.copy()
        prev_means = model.means_.copy()
        prev_covs = model.covariances_.copy()
        prev_pred = y_pred.copy()
        prev_ll = ll
    else:
        # Compare with previous
        weights_diff = np.abs(model.weights_ - prev_weights).max()
        means_diff = np.abs(model.means_ - prev_means).max()
        covs_diff = np.abs(model.covariances_ - prev_covs).max()
        pred_diff = np.abs(y_pred - prev_pred).max()
        ll_diff = abs(ll - prev_ll)
        
        print(f"\n=== COMPARISON WITH PREVIOUS ===")
        print(f"Weights difference: {weights_diff:.2e}")
        print(f"Means difference: {means_diff:.2e}")
        print(f"Covariances difference: {covs_diff:.2e}")
        print(f"Predictions difference: {pred_diff:.2e}")
        print(f"Log-likelihood difference: {ll_diff:.2e}")
        
        if weights_diff < 1e-10:
            print("🚨 BUG: Weights are identical!")
        if means_diff < 1e-10:
            print("🚨 BUG: Means are identical!")
        if covs_diff < 1e-10:
            print("🚨 BUG: Covariances are identical!")
        if pred_diff < 1e-10:
            print("🚨 BUG: Predictions are identical!")
        if ll_diff < 1e-10:
            print("🚨 BUG: Log-likelihood is identical!")

print(f"\n=== DIAGNOSIS ===")
print("If all differences are near zero, the discriminative EM is not working!")
print("The algorithm might be converging immediately without updating parameters.")


In [ ]:
# DEBUG: Test if random_state is causing identical results
print("\n=== DEBUG: TESTING RANDOM STATE EFFECT ===")

# Test with different random states
random_states = [42, 123, 456]

for i, random_state in enumerate(random_states):
    print(f"\n--- Random State {random_state} ---")
    
    model = DiscriminativeConditionalGMMRegressor(
        n_components=3,
        max_iter=50,
        tol=1e-4,
        weight_step=0.01,
        random_state=random_state
    )
    
    X_test = X[:100]
    y_test = y[:100]
    model.fit(X_test, y_test)
    
    print(f"Final weights: {model.weights_}")
    print(f"Final means[0]: {model.means_[0]}")
    
    y_pred = model.predict(X_test)
    ll = model.score(X_test, y_test)
    print(f"Log-likelihood: {ll:.6f}")
    
    if i == 0:
        prev_weights = model.weights_.copy()
        prev_means = model.means_.copy()
        prev_ll = ll
    else:
        weights_diff = np.abs(model.weights_ - prev_weights).max()
        means_diff = np.abs(model.means_ - prev_means).max()
        ll_diff = abs(ll - prev_ll)
        
        print(f"Difference from previous: weights={weights_diff:.2e}, means={means_diff:.2e}, ll={ll_diff:.2e}")
        
        if weights_diff < 1e-10:
            print("🚨 BUG: Different random states give identical weights!")
        if means_diff < 1e-10:
            print("🚨 BUG: Different random states give identical means!")
        if ll_diff < 1e-10:
            print("🚨 BUG: Different random states give identical log-likelihood!")

print(f"\n=== HYPOTHESIS ===")
print("If different random states give identical results, the discriminative EM is broken!")
print("The algorithm might not be using the random state properly or not updating parameters at all.")


In [ ]:
# DEBUG: Let's create a custom version that shows what's happening during training
print("\n=== DEBUGGING DISCRIMINATIVE EM ALGORITHM ===")

class DebugDiscriminativeConditionalGMMRegressor(DiscriminativeConditionalGMMRegressor):
    """Debug version that shows what's happening during training."""
    
    def fit(self, X, y):
        # Call parent fit but with debugging
        print(f"  Starting fit with n_components={self.n_components}, max_iter={self.max_iter}, tol={self.tol}, weight_step={self.weight_step}")
        
        # Validate and shape data
        X, y = validate_data(
            self,
            X,
            y,
            accept_sparse=False,
            y_numeric=True,
            multi_output=True,
        )
        y = np.asarray(y, dtype=float)
        if y.ndim == 1:
            y = y[:, None]

        n, Dx = X.shape
        Dy = y.shape[1]
        D = Dx + Dy
        self.n_features_in_ = Dx
        self.n_targets_ = Dy

        # Init by joint GMM on Z=[X|Y]
        Z = np.concatenate([X, y], axis=1)
        gmm = GaussianMixture(
            n_components=self.n_components,
            covariance_type=self.covariance_type,
            tol=self.tol,
            reg_covar=max(self.reg_covar, 1e-6),
            max_iter=self.max_iter,
            random_state=self.random_state,
            init_params=self.init_params,
        ).fit(Z)

        # Store joint params
        self.weights_ = gmm.weights_.astype(float, copy=True)
        self.means_ = gmm.means_.astype(float, copy=True)
        self.covariances_ = gmm.covariances_.astype(float, copy=True)

        print(f"  Initial weights: {self.weights_}")
        print(f"  Initial means[0]: {self.means_[0]}")

        # Ensure PD / positivity
        if self.covariance_type == "full":
            for k in range(self.n_components):
                self.covariances_[k] = self._ensure_positive_definite(self.covariances_[k])
        else:
            self.covariances_ = np.maximum(self.covariances_, self.reg_covar)

        # Initialize logits for weights
        self._alpha_ = np.log(self.weights_ + np.finfo(float).tiny)

        # Run discriminative EM with debugging
        prev_ll = -np.inf
        self.converged_ = False
        for it in range(1, self.max_iter + 1):
            prev_means = self.means_.copy()
            prev_covs = self.covariances_.copy()
            prev_weights = self.weights_.copy()

            r, s = self._e_step(X, y)
            self._m_step_joint(X, y, r)
            self._m_step_weights(r, s)

            # Check parameter changes
            means_delta = float(np.abs(self.means_ - prev_means).max())
            covs_delta = float(np.abs(self.covariances_ - prev_covs).max())
            weights_delta = float(np.abs(self.weights_ - prev_weights).max())
            param_delta = max(means_delta, covs_delta, weights_delta)
            
            if it <= 5 or it % 10 == 0:  # Show first 5 iterations and every 10th
                print(f"    Iter {it}: means_delta={means_delta:.2e}, covs_delta={covs_delta:.2e}, weights_delta={weights_delta:.2e}")
                print(f"      Weights: {self.weights_}")
            
            if param_delta < self.tol:
                self.converged_ = True
                print(f"    Converged at iteration {it} with param_delta={param_delta:.2e}")
                break

            ll = self._mean_conditional_loglik(X, y)
            prev_ll = ll

        self.lower_bound_ = float(self._mean_conditional_loglik(X, y))
        self.n_iter_ = int(it)
        print(f"  Final weights: {self.weights_}")
        print(f"  Final means[0]: {self.means_[0]}")
        print(f"  Final iterations: {self.n_iter_}, Converged: {self.converged_}")
        return self

# Test the debug version
print("Testing debug version:")
debug_model = DebugDiscriminativeConditionalGMMRegressor(
    n_components=3,
    max_iter=20,
    tol=1e-4,
    weight_step=0.01,
    random_state=42
)

X_debug = X[:50]  # Very small subset
y_debug = y[:50]
debug_model.fit(X_debug, y_debug)

ll_debug = debug_model.score(X_debug, y_debug)
print(f"Final log-likelihood: {ll_debug:.6f}")


In [ ]:
# Create time-based features for conditioning
df_clean['year'] = df_clean['datetime'].dt.year
df_clean['month'] = df_clean['datetime'].dt.month
df_clean['day_of_year'] = df_clean['datetime'].dt.dayofyear
df_clean['hour'] = df_clean['datetime'].dt.hour

# Create cyclical features
# Annual cycle (seasonal)
df_clean['annual_sin'] = np.sin(2 * np.pi * df_clean['day_of_year'] / 365.25)
df_clean['annual_cos'] = np.cos(2 * np.pi * df_clean['day_of_year'] / 365.25)

# Daily cycle (diurnal)
df_clean['daily_sin'] = np.sin(2 * np.pi * df_clean['hour'] / 24)
df_clean['daily_cos'] = np.cos(2 * np.pi * df_clean['hour'] / 24)

print("Created cyclical features:")
print("- Annual cycle: annual_sin, annual_cos")
print("- Daily cycle: daily_sin, daily_cos")


In [ ]:
# Transform positive variables using log(1+x) to handle zeros
df_clean['wind_ms_log'] = np.log1p(df_clean['wind_ms'])  # log(1 + wind_ms)
df_clean['ghi_wm2_log'] = np.log1p(df_clean['ghi_wm2'])  # log(1 + ghi_wm2)

# Prepare target variables (weather variables)
targets = ['temp_c', 'wind_ms_log', 'ghi_wm2_log']
y = df_clean[targets].values

# Prepare conditioning variables (time-based features)
conditioning_vars = ['annual_sin', 'annual_cos', 'daily_sin', 'daily_cos']
X = df_clean[conditioning_vars].values

print(f"Target variables: {targets}")
print(f"Conditioning variables: {conditioning_vars}")
print(f"X shape: {X.shape}, y shape: {y.shape}")

# Show transformation effect
print("\nTransformation comparison:")
print("Wind speed - Original vs Log-transformed:")
print(f"  Original: mean={df_clean['wind_ms'].mean():.2f}, std={df_clean['wind_ms'].std():.2f}")
print(f"  Log(1+x): mean={df_clean['wind_ms_log'].mean():.2f}, std={df_clean['wind_ms_log'].std():.2f}")
print("\nSolar radiation - Original vs Log-transformed:")
print(f"  Original: mean={df_clean['ghi_wm2'].mean():.2f}, std={df_clean['ghi_wm2'].std():.2f}")
print(f"  Log(1+x): mean={df_clean['ghi_wm2_log'].mean():.2f}, std={df_clean['ghi_wm2_log'].std():.2f}")


## Data Exploration

Let's explore the weather patterns and relationships before building our models.


In [ ]:
# Select samples to plot
df_sample = df_clean.sample(n=10000, random_state=42).sort_index()


# Create 3x3 grid: rows = variables, columns = time perspectives
fig, axes = plt.subplots(3, 3, figsize=(18, 12))

# Define colors for each variable
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green
variables = ['temp_c', 'wind_ms_log', 'ghi_wm2_log']
var_names = ['Temperature', 'Wind Speed (log)', 'Solar Radiation (log)']
units = ['°C', 'log(m/s)', 'log(W/m²)']

# Row 1: Temperature
# Hourly over time
axes[0, 0].plot(df_sample['datetime'], df_sample['temp_c'], alpha=0.7, linewidth=0.8, color=colors[0])
axes[0, 0].set_title('Temperature Over Time')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].tick_params(axis='x', rotation=45)

# vs Day of Year
axes[0, 1].scatter(df_sample['day_of_year'], df_sample['temp_c'], alpha=0.6, s=1, color=colors[0])
axes[0, 1].set_title('Temperature vs Day of Year')
axes[0, 1].set_xlabel('Day of Year')
axes[0, 1].set_ylabel('Temperature (°C)')

# vs Hour of Day
axes[0, 2].scatter(df_sample['hour'], df_sample['temp_c'], alpha=0.6, s=1, color=colors[0])
axes[0, 2].set_title('Temperature vs Hour of Day')
axes[0, 2].set_xlabel('Hour of Day')
axes[0, 2].set_ylabel('Temperature (°C)')
axes[0, 2].set_xticks(range(0, 24, 4))

# Row 2: Wind Speed
# Hourly over time
axes[1, 0].plot(df_sample['datetime'], df_sample['wind_ms_log'], alpha=0.7, linewidth=0.8, color=colors[1])
axes[1, 0].set_title('Wind Speed Over Time')
axes[1, 0].set_ylabel('Wind Speed (log)')
axes[1, 0].tick_params(axis='x', rotation=45)

# vs Day of Year
axes[1, 1].scatter(df_sample['day_of_year'], df_sample['wind_ms_log'], alpha=0.6, s=1, color=colors[1])
axes[1, 1].set_title('Wind Speed vs Day of Year')
axes[1, 1].set_xlabel('Day of Year')
axes[1, 1].set_ylabel('Wind Speed (log)')

# vs Hour of Day
axes[1, 2].scatter(df_sample['hour'], df_sample['wind_ms_log'], alpha=0.6, s=1, color=colors[1])
axes[1, 2].set_title('Wind Speed vs Hour of Day')
axes[1, 2].set_xlabel('Hour of Day')
axes[1, 2].set_ylabel('Wind Speed (log)')
axes[1, 2].set_xticks(range(0, 24, 4))

# Row 3: Solar Radiation
# Hourly over time
axes[2, 0].plot(df_sample['datetime'], df_sample['ghi_wm2_log'], alpha=0.7, linewidth=0.8, color=colors[2])
axes[2, 0].set_title('Solar Radiation Over Time')
axes[2, 0].set_ylabel('Solar Radiation (log)')
axes[2, 0].tick_params(axis='x', rotation=45)

# vs Day of Year
axes[2, 1].scatter(df_sample['day_of_year'], df_sample['ghi_wm2_log'], alpha=0.6, s=1, color=colors[2])
axes[2, 1].set_title('Solar Radiation vs Day of Year')
axes[2, 1].set_xlabel('Day of Year')
axes[2, 1].set_ylabel('Solar Radiation (log)')

# vs Hour of Day
axes[2, 2].scatter(df_sample['hour'], df_sample['ghi_wm2_log'], alpha=0.6, s=1, color=colors[2])
axes[2, 2].set_title('Solar Radiation vs Hour of Day')
axes[2, 2].set_xlabel('Hour of Day')
axes[2, 2].set_ylabel('Solar Radiation (log)')
axes[2, 2].set_xticks(range(0, 24, 4))

plt.tight_layout()
plt.show()

# Show some statistics about solar radiation
print("Solar Radiation Statistics:")
print(f"Original values - Min: {df_clean['ghi_wm2'].min():.1f}, Max: {df_clean['ghi_wm2'].max():.1f}, Mean: {df_clean['ghi_wm2'].mean():.1f}")
print(f"Log-transformed - Min: {df_clean['ghi_wm2_log'].min():.3f}, Max: {df_clean['ghi_wm2_log'].max():.3f}, Mean: {df_clean['ghi_wm2_log'].mean():.3f}")
print(f"Percentage of zero values: {(df_clean['ghi_wm2'] == 0).mean()*100:.1f}%")


In [ ]:
# Correlation analysis
fig, axes = plt.subplots(1, 2, figsize=(10, 8))

# Correlation matrix of weather variables
weather_vars = ['temp_c', 'wind_ms_log', 'ghi_wm2_log']
corr_matrix = df_sample[weather_vars].corr()

# Create heatmap using matplotlib
im = axes[0].imshow(corr_matrix.values, cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(corr_matrix.columns)))
axes[0].set_yticks(range(len(corr_matrix.index)))
axes[0].set_xticklabels(corr_matrix.columns, rotation=45)
axes[0].set_yticklabels(corr_matrix.index)
axes[0].set_title('Weather Variables Correlation')

# Add correlation values as text
for i in range(len(corr_matrix.index)):
    for j in range(len(corr_matrix.columns)):
        text = axes[0].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                           ha="center", va="center", color="black", fontweight='bold')


# Scatter plot matrix
from pandas.plotting import scatter_matrix
scatter_matrix(df_sample[weather_vars], alpha=0.6, ax=axes[1], diagonal='hist', hist_kwds={'bins':30})
axes[1].set_title('Weather Variables Scatter Matrix')

plt.tight_layout()
plt.show()

print("Correlation insights:")
print(f"Temperature vs Wind Speed: {corr_matrix.loc['temp_c', 'wind_ms_log']:.3f}")
print(f"Temperature vs Solar Radiation: {corr_matrix.loc['temp_c', 'ghi_wm2_log']:.3f}")
print(f"Wind Speed vs Solar Radiation: {corr_matrix.loc['wind_ms_log', 'ghi_wm2_log']:.3f}")


## Model Training

Now let's train different conditional mixture models to understand how weather variables depend on time patterns.


In [ ]:
# Use full dataset for training and analysis
X_train, y_train = X, y

print(f"Full dataset: {X_train.shape[0]} samples")
print(f"Training period: {df_clean.iloc[0]['datetime']} to {df_clean.iloc[-1]['datetime']}")
print("Using full dataset for both training and visualization to avoid artifacts from small test sets")


In [ ]:
# Train different models with better hyperparameters
n_components = 5

# Try different configurations to fix the issues
models = {
    'ConditionalGMM': ConditionalGMMRegressor(n_components=n_components, random_state=42, max_iter=200),
    'MixtureOfExperts': MixtureOfExpertsRegressor(
        n_components=n_components, 
        random_state=42, 
        max_iter=200,
        gating_max_iter=100,
        gating_tol=1e-4
    ),
    'Discriminative': DiscriminativeConditionalGMMRegressor(
        n_components=n_components, 
        random_state=42,
        max_iter=200,
        tol=1e-8,  # Much stricter tolerance to force more iterations
        weight_step=0.01  # Smaller step size for more stable convergence
    )
}

print("Training models...")
failed_models = []
for name, model in models.items():
    print(f"  Training {name}...")
    try:
        model.fit(X_train, y_train)
        
        # Check if model converged
        if hasattr(model, 'converged_'):
            print(f"    Converged: {model.converged_}")
        if hasattr(model, 'n_iter_'):
            print(f"    Iterations: {model.n_iter_}")
        
        # Evaluate performance on full dataset
        y_pred = model.predict(X_train)
        mse = mean_squared_error(y_train, y_pred)
        r2 = r2_score(y_train, y_pred)
        log_likelihood = model.score(X_train, y_train)
        
        # Check for NaN or infinite values
        nan_pred = np.isnan(y_pred).sum()
        inf_pred = np.isinf(y_pred).sum()
        nan_ll = np.isnan(log_likelihood) or np.isinf(log_likelihood)
        
        print(f"    MSE: {mse:.4f}, R²: {r2:.4f}, Log-likelihood: {log_likelihood:.4f}")
        if nan_pred > 0:
            print(f"    WARNING: {nan_pred} NaN predictions!")
        if inf_pred > 0:
            print(f"    WARNING: {inf_pred} infinite predictions!")
        if nan_ll:
            print(f"    WARNING: Invalid log-likelihood!")
            
        # Check prediction ranges
        print(f"    Prediction range: [{y_pred.min():.3f}, {y_pred.max():.3f}]")
        print(f"    Target range: [{y_train.min():.3f}, {y_train.max():.3f}]")
        
    except Exception as e:
        print(f"    ERROR: {e}")
        # Mark for removal
        failed_models.append(name)

# Remove failed models after iteration
for name in failed_models:
    del models[name]

print(f"\nTraining completed! {len(models)} models successful.")


In [ ]:
# Diagnostic analysis of model behavior
print("=== DIAGNOSTIC ANALYSIS ===")
print(f"Data shape: X={X_train.shape}, y={y_train.shape}")
print(f"X range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"y range: [{y_train.min():.3f}, {y_train.max():.3f}]")
print(f"X std: {X_train.std():.3f}")
print(f"y std: {y_train.std():.3f}")

# Check for any issues with the data
print(f"\nData quality checks:")
print(f"X has NaN: {np.isnan(X_train).any()}")
print(f"y has NaN: {np.isnan(y_train).any()}")
print(f"X has Inf: {np.isinf(X_train).any()}")
print(f"y has Inf: {np.isinf(y_train).any()}")

# Test each model with a small sample
print(f"\nTesting models with small sample:")
test_idx = np.random.choice(len(X_train), 100, replace=False)
X_test_small = X_train[test_idx]
y_test_small = y_train[test_idx]

# Create a list of model names to avoid dictionary modification during iteration
model_names = list(models.keys())

for name in model_names:
    if name not in models:  # Skip if model was removed
        continue
        
    print(f"\n{name}:")
    try:
        model = models[name]
        
        # Test prediction
        y_pred_small = model.predict(X_test_small)
        print(f"  Prediction shape: {y_pred_small.shape}")
        print(f"  Prediction range: [{y_pred_small.min():.3f}, {y_pred_small.max():.3f}]")
        
        # Test sampling
        samples = model.sample(X_test_small[:5], n_samples=3)
        print(f"  Sample shape: {samples.shape}")
        print(f"  Sample range: [{samples.min():.3f}, {samples.max():.3f}]")
        
        # Test log_prob
        log_probs = model.log_prob(X_test_small[:5], y_test_small[:5])
        print(f"  Log prob shape: {log_probs.shape}")
        print(f"  Log prob range: [{log_probs.min():.3f}, {log_probs.max():.3f}]")
        
        # Test log-likelihood (score method)
        log_likelihood = model.score(X_test_small, y_test_small)
        print(f"  Log-likelihood: {log_likelihood:.4f}")
        
        # Check for NaN or infinite values in log-likelihood
        if np.isnan(log_likelihood) or np.isinf(log_likelihood):
            print(f"  WARNING: Invalid log-likelihood!")
        
    except Exception as e:
        print(f"  ERROR: {e}")

print("\n=== END DIAGNOSTIC ===")


In [ ]:
# Specific investigation of Discriminative model issues
print("=== DISCRIMINATIVE MODEL INVESTIGATION ===")

if 'Discriminative' in models:
    disc_model = models['Discriminative']
    
    # Check model parameters
    print("Model parameters:")
    print(f"  n_components: {disc_model.n_components}")
    print(f"  covariance_type: {disc_model.covariance_type}")
    print(f"  max_iter: {disc_model.max_iter}")
    print(f"  tol: {disc_model.tol}")
    
    # Check fitted parameters
    if hasattr(disc_model, 'weights_'):
        print(f"  Weights shape: {disc_model.weights_.shape}")
        print(f"  Weights range: [{disc_model.weights_.min():.6f}, {disc_model.weights_.max():.6f}]")
        print(f"  Weights sum per component: {disc_model.weights_.sum(axis=0)}")
    
    if hasattr(disc_model, 'means_'):
        print(f"  Means shape: {disc_model.means_.shape}")
        print(f"  Means range: [{disc_model.means_.min():.3f}, {disc_model.means_.max():.3f}]")
    
    if hasattr(disc_model, 'covariances_'):
        print(f"  Covariances shape: {disc_model.covariances_.shape}")
        print(f"  Covariances range: [{disc_model.covariances_.min():.6f}, {disc_model.covariances_.max():.6f}]")
    
    # Test with different parameters
    print("\nTesting with different parameters:")
    test_params = [
        {'n_components': 3, 'max_iter': 50, 'tol': 1e-3},
        {'n_components': 2, 'max_iter': 100, 'tol': 1e-4},
        {'n_components': 4, 'max_iter': 200, 'tol': 1e-5}
    ]
    
    for i, params in enumerate(test_params):
        print(f"\nTest {i+1}: {params}")
        try:
            test_model = DiscriminativeConditionalGMMRegressor(**params, random_state=42)
            test_model.fit(X_train[:1000], y_train[:1000])  # Use smaller sample for speed
            
            y_pred_test = test_model.predict(X_train[:100])
            mse_test = mean_squared_error(y_train[:100], y_pred_test)
            r2_test = r2_score(y_train[:100], y_pred_test)
            
            print(f"  MSE: {mse_test:.4f}, R²: {r2_test:.4f}")
            print(f"  Converged: {getattr(test_model, 'converged_', 'Unknown')}")
            
        except Exception as e:
            print(f"  ERROR: {e}")

print("\n=== END DISCRIMINATIVE INVESTIGATION ===")


In [ ]:
# Deep dive into Discriminative model convergence issues
print("=== DISCRIMINATIVE CONVERGENCE INVESTIGATION ===")

# Test with a very simple case first
print("Testing with simple 2D case (temperature only):")
X_simple = X_train[:, :2]  # Only annual and daily cycles
y_simple = y_train[:, 0:1]  # Only temperature

print(f"Simple data shape: X={X_simple.shape}, y={y_simple.shape}")

# Test different configurations with much stricter tolerances
test_configs = [
    {'n_components': 3, 'max_iter': 100, 'tol': 1e-8, 'weight_step': 0.01},
    {'n_components': 2, 'max_iter': 200, 'tol': 1e-9, 'weight_step': 0.005},
    {'n_components': 4, 'max_iter': 300, 'tol': 1e-10, 'weight_step': 0.02},
    {'n_components': 3, 'max_iter': 150, 'tol': 1e-7, 'weight_step': 0.01}
]

for i, config in enumerate(test_configs):
    print(f"\nConfig {i+1}: {config}")
    try:
        test_model = DiscriminativeConditionalGMMRegressor(**config, random_state=42)
        test_model.fit(X_simple[:1000], y_simple[:1000])  # Use smaller sample
        
        print(f"  Converged: {getattr(test_model, 'converged_', 'Unknown')}")
        print(f"  Iterations: {getattr(test_model, 'n_iter_', 'Unknown')}")
        
        if hasattr(test_model, 'weights_'):
            print(f"  Weights: {test_model.weights_[:3]}")  # Show first 3 weights
        
        # Test prediction
        y_pred_test = test_model.predict(X_simple[:10])
        print(f"  Prediction range: [{y_pred_test.min():.3f}, {y_pred_test.max():.3f}]")
        
        # Test log-likelihood
        log_likelihood = test_model.score(X_simple[:100], y_simple[:100])
        print(f"  Log-likelihood: {log_likelihood:.4f}")
        
        # Check for invalid log-likelihood
        if np.isnan(log_likelihood) or np.isinf(log_likelihood):
            print(f"  WARNING: Invalid log-likelihood!")
        
    except Exception as e:
        print(f"  ERROR: {e}")

print("\n=== END CONVERGENCE INVESTIGATION ===")


In [ ]:
# Investigate MoE temperature prediction issues
print("=== MOE TEMPERATURE INVESTIGATION ===")

if 'MixtureOfExperts' in models:
    moe_model = models['MixtureOfExperts']
    
    print("MoE Model Analysis:")
    print(f"  n_components: {moe_model.n_components}")
    print(f"  max_iter: {moe_model.max_iter}")
    print(f"  converged: {getattr(moe_model, 'converged_', 'Unknown')}")
    print(f"  iterations: {getattr(moe_model, 'n_iter_', 'Unknown')}")
    
    # Check the internal parameters
    if hasattr(moe_model, '_params'):
        params = moe_model._params
        print(f"  Expert A shape: {params.A.shape if params.A is not None else 'None'}")
        if params.A is not None:
            print(f"  Expert A range: [{params.A.min():.3f}, {params.A.max():.3f}]")
        print(f"  Expert b shape: {params.b.shape}")
        print(f"  Expert b range: [{params.b.min():.3f}, {params.b.max():.3f}]")
        print(f"  Gating W shape: {params.W.shape}")
        print(f"  Gating W range: [{params.W.min():.3f}, {params.W.max():.3f}]")
        print(f"  Gating c shape: {params.c.shape}")
        print(f"  Gating c range: [{params.c.min():.3f}, {params.c.max():.3f}]")
    
    # Test predictions on different time points
    print("\nTesting predictions on different time points:")
    test_times = [
        ("Winter midnight", 355, 0),    # Winter midnight
        ("Summer noon", 172, 12),       # Summer noon
        ("Spring dawn", 80, 6),         # Spring dawn
    ]
    
    for name, day_of_year, hour in test_times:
        X_test_time = np.array([[
            np.sin(2 * np.pi * day_of_year / 365.25),
            np.cos(2 * np.pi * day_of_year / 365.25),
            np.sin(2 * np.pi * hour / 24),
            np.cos(2 * np.pi * hour / 24)
        ]])
        
        y_pred_time = moe_model.predict(X_test_time)
        print(f"  {name}: {y_pred_time[0]}")
        
        # Check gating probabilities
        if hasattr(moe_model, '_compute_conditional_mixture'):
            mixture_params = moe_model._compute_conditional_mixture(X_test_time)
            weights = mixture_params['weights'][0]
            print(f"    Gating weights: {weights[:5]}")  # Show first 5 weights

print("\n=== END MOE INVESTIGATION ===")


In [ ]:
# Systematic convergence testing
print("=== SYSTEMATIC CONVERGENCE TESTING ===")

# Test with very strict tolerances to force more iterations
tolerance_tests = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]

print("Testing different tolerance values:")
for tol in tolerance_tests:
    print(f"\nTolerance: {tol}")
    try:
        test_model = DiscriminativeConditionalGMMRegressor(
            n_components=5,
            random_state=42,
            max_iter=500,  # High max_iter to see actual convergence
            tol=tol,
            weight_step=0.01
        )
        test_model.fit(X_train[:2000], y_train[:2000])  # Use subset for speed
        
        iterations = getattr(test_model, 'n_iter_', 'Unknown')
        converged = getattr(test_model, 'converged_', 'Unknown')
        log_likelihood = test_model.score(X_train[:100], y_train[:100])
        
        print(f"  Iterations: {iterations}")
        print(f"  Converged: {converged}")
        print(f"  Log-likelihood: {log_likelihood:.4f}")
        
        # Check if we hit max_iter
        if iterations == 500:
            print(f"  WARNING: Hit max_iter limit!")
            
    except Exception as e:
        print(f"  ERROR: {e}")

print("\n=== END CONVERGENCE TESTING ===")


## Model Analysis and Visualization

Let's analyze how well our models capture the weather patterns and create visualizations of the learned distributions.


In [ ]:
# Compare model performance
results = []
for name, model in models.items():
    y_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    log_likelihood = model.score(X_train, y_train)
    
    results.append({
        'Model': name,
        'MSE': mse,
        'R²': r2,
        'Log-Likelihood': log_likelihood
    })

results_df = pd.DataFrame(results)
print("Model Performance Comparison:")
print(results_df.round(4))

# Visualize performance
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['MSE', 'R²', 'Log-Likelihood']
for i, metric in enumerate(metrics):
    axes[i].bar(results_df['Model'], results_df[metric], alpha=0.7)
    axes[i].set_title(f'{metric} Comparison')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Analyze seasonal patterns using the best model
best_model_name = results_df.loc[results_df['Log-Likelihood'].idxmax(), 'Model']
best_model = models[best_model_name]
print(f"Best model: {best_model_name}")

# Create a grid of time points for analysis
days_of_year = np.linspace(1, 365, 12)  # 12 points per year
hours_of_day = [6, 12, 18, 0]  # Dawn, noon, dusk, midnight

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, hour in enumerate(hours_of_day):
    # Create conditioning variables for this time
    X_condition = np.zeros((len(days_of_year), 4))
    X_condition[:, 0] = np.sin(2 * np.pi * days_of_year / 365.25)  # annual_sin
    X_condition[:, 1] = np.cos(2 * np.pi * days_of_year / 365.25)  # annual_cos
    X_condition[:, 2] = np.sin(2 * np.pi * hour / 24)  # daily_sin
    X_condition[:, 3] = np.cos(2 * np.pi * hour / 24)  # daily_cos
    
    # Get predictions
    y_pred = best_model.predict(X_condition)
    
    # Plot temperature and wind speed
    ax = axes[i]
    ax2 = ax.twinx()
    
    line1 = ax.plot(days_of_year, y_pred[:, 0], 'b-', linewidth=2, label='Temperature')
    line2 = ax2.plot(days_of_year, y_pred[:, 1], 'r-', linewidth=2, label='Wind Speed (log)')
    
    ax.set_xlabel('Day of Year')
    ax.set_ylabel('Temperature (°C)', color='b')
    ax2.set_ylabel('Wind Speed (log)', color='r')
    ax.set_title(f'Hour {hour:02d}:00 - Seasonal Patterns')
    
    # Add month labels
    month_days = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ax.set_xticks(month_days)
    ax.set_xticklabels(month_names)
    
    # Combine legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
# Analyze joint densities at specific time points
time_points = [
    ('Winter Solstice (Dec 21, 12:00)', 355, 12),  # Winter noon
    ('Spring Equinox (Mar 21, 12:00)', 80, 12),    # Spring noon
    ('Summer Solstice (Jun 21, 12:00)', 172, 12),  # Summer noon
    ('Autumn Equinox (Sep 21, 12:00)', 264, 12)   # Autumn noon
]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, (title, day_of_year, hour) in enumerate(time_points):
    # Create conditioning variables
    X_condition = np.array([[
        np.sin(2 * np.pi * day_of_year / 365.25),  # annual_sin
        np.cos(2 * np.pi * day_of_year / 365.25),  # annual_cos
        np.sin(2 * np.pi * hour / 24),             # daily_sin
        np.cos(2 * np.pi * hour / 24)              # daily_cos
    ]])
    
    # Get conditional mixture
    gmm = best_model.condition(X_condition)
    
    # Generate samples from the conditional distribution
    n_samples = 1000
    samples = gmm.sample(n_samples)[0]  # (n_samples, 3)
    
    # Create a grid for density calculation
    temp_range = np.linspace(samples[:, 0].min() - 2, samples[:, 0].max() + 2, 50)
    wind_range = np.linspace(samples[:, 1].min() - 0.5, samples[:, 1].max() + 0.5, 50)
    temp_grid, wind_grid = np.meshgrid(temp_range, wind_range)
    
    # Calculate density for each grid point
    grid_points = np.column_stack([temp_grid.ravel(), wind_grid.ravel()])
    
    # Create dummy solar radiation values for the grid (we'll use the mean)
    solar_mean = samples[:, 2].mean()
    grid_solar = np.full((grid_points.shape[0],), solar_mean)
    
    # Calculate log probabilities
    log_probs = gmm.score_samples(np.column_stack([grid_points, grid_solar.reshape(-1, 1)]))
    density = np.exp(log_probs).reshape(temp_grid.shape)
    
    # Plot temperature vs wind speed
    ax = axes[i]
    
    # Plot density contours
    contour = ax.contour(temp_grid, wind_grid, density, levels=8, alpha=0.7, colors='black', linewidths=1)
    ax.clabel(contour, inline=True, fontsize=8, fmt='%.3f')
    
    # Plot samples with solar radiation color
    scatter = ax.scatter(samples[:, 0], samples[:, 1], alpha=0.6, s=20, c=samples[:, 2], 
                        cmap='viridis', label='Solar Radiation (log)')
    
    ax.set_xlabel('Temperature (°C)')
    ax.set_ylabel('Wind Speed (log)')
    ax.set_title(f'{title}\nTemperature vs Wind Speed (with PDF contours)')
    
    # Add colorbar for solar radiation
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Solar Radiation (log)')
    
    # Add some statistics
    temp_mean = samples[:, 0].mean()
    wind_mean = samples[:, 1].mean()
    ax.text(0.05, 0.95, f'Mean Temp: {temp_mean:.1f}°C\nMean Wind: {wind_mean:.2f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()


In [ ]:
# Analyze diurnal patterns (daily cycles)
hours = np.arange(0, 24)
day_of_year = 172  # Summer solstice

# Create conditioning variables for each hour
X_condition = np.zeros((24, 4))
X_condition[:, 0] = np.sin(2 * np.pi * day_of_year / 365.25)  # annual_sin
X_condition[:, 1] = np.cos(2 * np.pi * day_of_year / 365.25)  # annual_cos
X_condition[:, 2] = np.sin(2 * np.pi * hours / 24)  # daily_sin
X_condition[:, 3] = np.cos(2 * np.pi * hours / 24)  # daily_cos

# Get predictions
y_pred = best_model.predict(X_condition)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Temperature throughout the day
axes[0, 0].plot(hours, y_pred[:, 0], 'b-', linewidth=2, marker='o')
axes[0, 0].set_title('Temperature Throughout the Day (Summer Solstice)')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].grid(True, alpha=0.3)

# Wind speed throughout the day
axes[0, 1].plot(hours, y_pred[:, 1], 'r-', linewidth=2, marker='o')
axes[0, 1].set_title('Wind Speed Throughout the Day (Summer Solstice)')
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Wind Speed (log)')
axes[0, 1].grid(True, alpha=0.3)

# Solar radiation throughout the day
axes[1, 0].plot(hours, y_pred[:, 2], 'g-', linewidth=2, marker='o')
axes[1, 0].set_title('Solar Radiation Throughout the Day (Summer Solstice)')
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('Solar Radiation (log)')
axes[1, 0].grid(True, alpha=0.3)

# All variables together
ax = axes[1, 1]
ax2 = ax.twinx()
ax3 = ax.twinx()
ax3.spines['right'].set_position(('outward', 60))

line1 = ax.plot(hours, y_pred[:, 0], 'b-', linewidth=2, label='Temperature')
line2 = ax2.plot(hours, y_pred[:, 1], 'r-', linewidth=2, label='Wind Speed')
line3 = ax3.plot(hours, y_pred[:, 2], 'g-', linewidth=2, label='Solar Radiation')

ax.set_xlabel('Hour of Day')
ax.set_ylabel('Temperature (°C)', color='b')
ax2.set_ylabel('Wind Speed (log)', color='r')
ax3.set_ylabel('Solar Radiation (log)', color='g')
ax.set_title('All Variables Throughout the Day')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Analyze uncertainty in predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

for i, (name, model) in enumerate(models.items()):
    # Generate samples for uncertainty analysis (use first 100 samples from full dataset)
    n_samples = 1000
    samples = model.sample(X_train[:100], n_samples=n_samples)  # (100, n_samples, 3)
    
    # Calculate prediction intervals
    temp_mean = samples[:, :, 0].mean(axis=1)
    temp_std = samples[:, :, 0].std(axis=1)
    temp_lower = np.percentile(samples[:, :, 0], 2.5, axis=1)
    temp_upper = np.percentile(samples[:, :, 0], 97.5, axis=1)
    
    # Plot temperature predictions with uncertainty
    ax = axes[i]
    test_indices = range(len(temp_mean))
    
    ax.fill_between(test_indices, temp_lower, temp_upper, alpha=0.3, label='95% Prediction Interval')
    ax.plot(test_indices, temp_mean, 'b-', linewidth=2, label='Mean Prediction')
    ax.plot(test_indices, y_train[:100, 0], 'r--', linewidth=1, label='Actual')
    
    ax.set_title(f'{name} - Temperature Predictions')
    ax.set_xlabel('Test Sample Index')
    ax.set_ylabel('Temperature (°C)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
